# Проект по исследованию базы данных сервиса для чтения книг по подписке

<font size = 5 > Цель проекта: </font> <div>  исследование существующей базы данных сервиса для чтения книг по подписке для поиска ценностного предложения для нового продукта <div>
<br>
<font size = 5 >  Описание данных: </font> <div>

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.
    
<br>
<font size = 5 >  Задания: </font> <div>    

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.    
    

### Загрузка библиотек, подключение к базе, исследование таблиц 

In [21]:
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [22]:
query = """
SELECT *
FROM books
LIMIT 5;
"""
books = pd.io.sql.read_sql(query, con = engine)
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [23]:
query = """
SELECT *
FROM authors
LIMIT 5;
"""
authors = pd.io.sql.read_sql(query, con = engine)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [24]:
query = """
SELECT *
FROM publishers
LIMIT 5;
"""
publishers = pd.io.sql.read_sql(query, con = engine)
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [25]:
query = """
SELECT *
FROM ratings
LIMIT 5;
"""
ratings = pd.io.sql.read_sql(query, con = engine)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [26]:
query = """
SELECT *
FROM reviews
LIMIT 5;
"""
reviews = pd.io.sql.read_sql(query, con = engine)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Посчитайте, сколько книг вышло после 1 января 2000 года;

In [27]:
query = """
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'::date

"""
books_cnt = pd.io.sql.read_sql(query, con = engine)
books_cnt


,count
0,819


* После 01 янвря вышло 819 книг

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [28]:
query = """
SELECT 
    b.title, 
    COUNT(DISTINCT rv.review_id) AS reviews_count, 
    AVG(r.rating) AS average_rating
FROM 
    books b
    LEFT JOIN ratings r ON b.book_id = r.book_id
    LEFT JOIN reviews rv ON b.book_id = rv.book_id
GROUP BY 
    b.book_id
ORDER BY reviews_count DESC
LIMIT 10;
"""
books_avg = pd.io.sql.read_sql(query, con = engine)
books_avg

,title,reviews_count,average_rating
0,Twilight (Twilight #1),7,3.662500
1,The Da Vinci Code (Robert Langdon #2),6,3.830508
2,Eat Pray Love,6,3.395833
3,The Alchemist,6,3.789474
4,The Catcher in the Rye,6,3.825581
5,The Curious Incident of the Dog in the Night-Time,6,4.081081
6,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
7,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
8,Outlander (Outlander #1),6,4.125000
9,The Book Thief,6,4.264151


**Сумерки )**

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [29]:
query = """
WITH publisher_books_cnt AS (
    SELECT publisher,
           COUNT(book_id) AS books_cnt
    FROM publishers AS p
    INNER JOIN books AS b 
    ON p.publisher_id = b.publisher_id
    WHERE num_pages > 50
    GROUP BY publisher
    ORDER BY books_cnt DESC
    )
SELECT publisher
FROM publisher_books_cnt
WHERE books_cnt = (SELECT MAX(books_cnt)
                   FROM publisher_books_cnt)
"""
books_pub = pd.io.sql.read_sql(query, con = engine)
books_pub

,publisher
0,Penguin Books


<div class="alert alert-success">
<b>Комментарий ревьюера:</b>☀️
    
Чемпион определен верно 

Наш чемпион **Penguin Books**

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [30]:
query = """

SELECT author,
       AVG(rating) AS mean_rating
FROM authors AS a
     JOIN books AS b 
     ON a.author_id = b.author_id
     JOIN ratings AS r 
     ON b.book_id = r.book_id

WHERE b.book_id IN (
                  SELECT book_id
                  FROM ratings 
                  GROUP BY book_id
                  HAVING COUNT(rating_id) >= 50
                  ORDER BY COUNT(rating_id) DESC)
GROUP BY author
ORDER BY mean_rating DESC
   LIMIT 1

"""
top_auhtor = pd.io.sql.read_sql(query, con = engine)
top_auhtor

,author,mean_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Самая выская оценка у Автора .K. Rowling/Mary GrandPré	

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [31]:
query = """
SELECT ROUND(AVG(review_cnt))
FROM (
       SELECT username, 
              COUNT(review_id) AS review_cnt
       FROM reviews 
       WHERE username IN (
                          SELECT username
                          FROM ratings
                          GROUP BY username
                          HAVING COUNT(rating_id) > 50)
       GROUP BY  username
      ) AS rereview_count
"""
top_50= pd.io.sql.read_sql(query, con = engine)
top_50

,round
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок- 24

# Вывод

* После 1 января вышло 819 книг  
* Больше всего обзоров у Сумерек и средней оценкой 3.6  
* Издательство которое выпустило больше всего книг тольще 50 стр Penguin Books
* Автор с самой высокой средней оценкой Роулинг  
* Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок- 24.